In [ ]:

from utilities import resize
resize(100)

#!pip install "tensorflow==2.8.*"
!pip install fasttext > /dev/null
!pip install "tensorflow-text==2.8.*" > /dev/null   #TODO CHECK
!pip install datasets > /dev/null
!pip install sacrebleu > /dev/null
!pip install sacremoses > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
try:
    from keras.preprocessing.sequence import pad_sequences
except ImportError:
    from keras.utils import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from collections import OrderedDict
import nltk
from nltk.tokenize import word_tokenize
import fasttext
import fasttext.util
import matplotlib.pyplot as plt
import pickle 
import typing
import random
from datasets import load_metric
from typing import Any, Tuple
import tensorflow_text as tf_text
from os.path import join, dirname
try:
    from google.colab import drive
    COLAB = True
except ModuleNotFoundError:
    COLAB = False

from data_cleaning import clean_data
from data_preparation import build_vocabulary
    
from utilities import memoization, plot_trend

#tf.config.run_functions_eagerly(True)

@memoization(0)
def word_tk_mem(arg):
    return word_tokenize(arg)

nltk.download('punkt')
if COLAB:
    drive.mount('/content/drive')
    PATH = '/content/drive/MyDrive/UniBO/NLP'
else:
    PATH = './'
use_builtins = True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [ ]:
df = clean_data(PATH)
df.dropna(inplace=True)
df.head(15)

/content/drive/MyDrive/UniBO/NLP
DF 1 has shape (63006, 2)
DF 2 has shape (1166, 2)


,sentence_1,sentence_2
0,ma questo a cosa servirebbe ?,a che servono queste cose ?
1,"salve, avrei bisogno di una informazione piutt...",ho bisogno di una informazione urgente .
2,ciao a tutti avrei bisogno di un consiglio .,ho bisogno di un suo consiglio .
3,possibilmente uno che avesse bisogno dell aiuto .,ho bisogno di un vostro aiuto .
4,questa sarebbe una cosa positiva.,questa era una nuova cosa .
5,possibilmente uno che avesse bisogno dell aiuto .,ho bisogno di un aiuto .
6,avrei bisogno di un aiuto .,ho bisogno di un vostro aiuto .
7,quale sarebbe allora la soluzione giusta ?,è questa la soluzione giusta ?
8,"salve , avrei bisogno di un vostro aiuto .",abbiamo bisogno del vostro aiuto .
9,"salve , avrei bisogno di un vostro aiuto .",ho bisogno del vostro aiuto .


https://drive.google.com/file/d/1C-CBzudbdaDrClPwTwqEgQVCP-Af40Za/view?usp=sharing

https://drive.google.com/file/d/1azLhm1to1JWiKOLdaZ3NQ2iouKnigVyD/view?usp=sharing

In [ ]:
!cp /content/drive/MyDrive/paisa_clean_95_th.txt ./paisa_clean_95_th.txt 
!cp /content/drive/MyDrive/paisa_small.txt ./paisa_small.txt #subset of paisa with only 100000 sentences 

In [ ]:
MAX_SEQ_LENGHT=111 #obtained from texting covers 95th percentile of the data in the full paisa dataset 
TRAIN_DF, TEST_DF = train_test_split(df, test_size=0.3)
BATCH_SIZE = 64

def format_dataset(cplx, simpl):
  return ({"encoder_inputs": cplx, "decoder_inputs": simpl[:, :-1],}, simpl[:, 1:])

TRAIN_DATASET = tf.data.TextLineDataset(['paisa_clean_95_th.txt']).batch(BATCH_SIZE).shuffle(2048).prefetch(16).cache()
#


In [ ]:

VOCABULARY_SIZE=20000+2 #+2 one for unknown one for empty string(padding)

def cr_vocab_vector(text_dataset):
  def tf_lower_and_split_punct(text): 
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    text = tf.strings.join(['[START]',text, '[END]'], separator=' ')
    return text
  Vec_layer=tf.keras.layers.TextVectorization(standardize=tf_lower_and_split_punct,max_tokens=VOCABULARY_SIZE,output_sequence_length=MAX_SEQ_LENGHT+2)
  Vec_layer.adapt(text_dataset)#vocabulary generation using the largest dataset more statistical significance
  return Vec_layer.get_vocabulary(),Vec_layer

IDX_TO_WORD,VECTORIZER=cr_vocab_vector(TRAIN_DATASET) 





In [ ]:
WORD_LENGTH=[len(i) for i in IDX_TO_WORD]


In [ ]:
#sentence_1=[VECTORIZER(i) for  i in TRAIN_DF['sentence_1']]
#sentence_2=[VECTORIZER(i) for  i in TRAIN_DF['sentence_2']] #used for testing no longer needed 

#TRAIN_DATASET =tf.data.Dataset.from_tensor_slices((sentence_1,sentence_2))

In [ ]:
sentence_1=[VECTORIZER(i) for  i in TEST_DF['sentence_1']]
sentence_2=[VECTORIZER(i) for  i in TEST_DF['sentence_2']]

TEST_DATASET =tf.data.Dataset.from_tensor_slices((sentence_1,sentence_2)).batch(BATCH_SIZE).shuffle(2048).prefetch(16).cache()

In [ ]:
#TRAIN_DATASET =TRAIN_DATASET.batch(64).shuffle(2048).prefetch(16).cache() #testing

In [ ]:
TRAIN_DATASET = tf.data.TextLineDataset(['/content/paisa_small.txt']).batch(BATCH_SIZE).shuffle(2048).prefetch(16).cache()


In [ ]:
EMBEDD_DIM = 200

EMBEDD_FNAME = f'cc.it.{EMBEDD_DIM}.bin'  #ex. 'cc.it.100.bin'

try: EMBEDDING_MODEL
except NameError:
    # Downloading FastText italian pre-trained model
    fasttext.FastText.eprint = lambda x: None
    try:
        print('Loading fasttext model...', flush=True)
        EMBEDDING_MODEL = fasttext.load_model(join(PATH, EMBEDD_FNAME))
    except ValueError as e:
        print(e)
        fasttext.util.download_model('it', if_exists='ignore')  
        EMBEDDING_MODEL = fasttext.load_model(join(PATH, EMBEDD_FNAME))

print(EMBEDDING_MODEL.get_dimension())

Loading fasttext model...


In [ ]:
EMBEDDING_DIM = EMBEDDING_MODEL.get_dimension()
def compute_embedding_matrix(embedding_model, idx_to_word):
  """
  Return the embedding matrix of the train vocabulary:
  """
  embedding_matrix = np.zeros((VOCABULARY_SIZE, EMBEDDING_DIM), dtype='float32')

  for idx,word in enumerate(idx_to_word):
    embedding_matrix[idx] = embedding_model.get_word_vector(word)
  return embedding_matrix

EMBEDDING_MATRIX = compute_embedding_matrix(EMBEDDING_MODEL, IDX_TO_WORD)

In [ ]:
END_NUMBER=VECTORIZER('[END]')[2]
PAD_NUMBER=VECTORIZER('')[-1]
START_NUMBER=VECTORIZER('[Start]')[0]

In [ ]:
WORD_LENGTH[END_NUMBER]=0 #non words are considered of lenght zero
WORD_LENGTH[PAD_NUMBER]=0
WORD_LENGTH[START_NUMBER]=0

[END]

[START]


In [ ]:
IDX_TO_WORD=tf.constant(IDX_TO_WORD) 
EMBEDDING_MATRIX =tf.constant(EMBEDDING_MATRIX ) #tensor are easier to manipulate

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, initializer, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        

        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim, embeddings_initializer=initializer, mask_zero=True
        )
        self.token_embeddings.trainable=False
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:
WORD_LENGTH[1]=sum(WORD_LENGTH[2:])/(len(WORD_LENGTH)-4) #lenght of unknown token is evalueted as the mean of the lenghts of all words in the vocabulary

WORD_LENGTH=tf.constant(WORD_LENGTH)


In [ ]:
class GULPEASE_index(tf.keras.losses.Loss):
  def __init__(self):
    self.name='gulpease_index'
  def __call__(self,to_cast,y_pred):
    """
    https://it.wikipedia.org/wiki/Indice_Gulpease
    the implementation is impossible but if we take the lenght o  all the words in the vocabulary 
    and multiply by the probability of each word we have a possible representation of the number of 
    characters per sentence
    the number of words can  be inferred as max -number of non words.
    """
    global IDX_TO_WORD,END_NUMBER
    #removing all zero length words from the total
    n_words=MAX_SEQ_LENGHT-tf.reduce_sum(y_pred[:,:,END_NUMBER],axis=1)-tf.reduce_sum(y_pred[:,:,PAD_NUMBER],axis=1)-tf.reduce_sum(y_pred[:,:,START_NUMBER],axis=1)
    tot_char=tf.math.reduce_sum(y_pred*WORD_LENGTH,axis=[1,2])
    return tf.reduce_mean((89 +(300 -tot_char*10)/n_words)/100)






class Simplification_loss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'simplification_loss'
    self.loss = tf.keras.losses.CosineSimilarity()
    self.gulpease=GULPEASE_index()
  def __call__(self, input,y_pred):
    """
    multiply the embedding matrix for the output this is the 
    """
    global EMBEDDING_MATRIX

   
    # Calculate the loss for each item in the batch.
    gulpease=self.gulpease(None,y_pred)
    gulpease=10*(0.7-gulpease)*(0.7-gulpease)
    input=tf.cast(input,tf.int32)
    embd_inp=tf.gather_nd(EMBEDDING_MATRIX,tf.expand_dims(input,-1))
    embd_out=tf.reduce_mean(y_pred,axis=1)
    embd_out=tf.expand_dims(embd_out,axis=-1)*EMBEDDING_MATRIX
    

    embd=tf.reduce_mean(embd_inp,axis=1)
    #only padding is removed in the count of words the other non words have non zero embedding 
    n_words=MAX_SEQ_LENGHT-tf.reduce_sum(y_pred[:,:,PAD_NUMBER],axis=1) 
   
    mean_input=embd/tf.expand_dims(tf.reduce_sum(tf.cast(input!= 0,tf.float32),axis=1),axis=-1)

    mean_output=tf.reduce_sum(embd_out,axis=1)/tf.expand_dims(n_words,axis=-1)
    # Mask off the losses on padding.
    
    # Return the total.
    return tf.cast(1,tf.float32)+self.loss(mean_input,mean_output)-gulpease




def return_loss():
  simp_loss=Simplification_loss()
  def custom_loss(y_true, y_pred):
    weighted_loss = simp_loss(y_true, y_pred) # your implementation of weighted crossentropy loss
    #unweighted_loss = K.sparse_categorical_crossentropy(y_true, y_pred)
    return weighted_loss#K.in_train_phase(weighted_loss, unweighted_loss)
  return custom_loss
def return_accuracy():
  accuracy=tf.keras.metrics.Accuracy()

  def custom_accuracy(y_true, y_pred):
    train_acc=0
    test_acc=accuracy(y_true, y_pred)
    return K.in_train_phase(train_acc,test_acc) 
  return custom_accuracy


class CustomModel(keras.Model):
  def train_step(self, data):
    global VECTORIZER
    x=VECTORIZER(data)
   
    with tf.GradientTape() as tape:
      y_pred=self([x,x], training=True)
        
      loss = self.compiled_loss(x,y_pred)
   ion layer by passing a list of vocabulary terms to the layer's __init__() meth trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    # Update weights
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # Update metrics (includes the metric that tracks the loss)
    self.compiled_metrics.update_state(x,y_pred)
    return{'loss':loss}#{m.name: m.result() for m in self.metrics}
  def test_step(self, data):
    # Unpack the data
    x,y=data
    x=tf.cast(x,tf.int32)
    y=tf.cast(y,tf.int32)
    cce = tf.keras.losses.CategoricalCrossentropy()
    # Compute predictions
    y_pred = self([x,x], training=False)
    # Updates the metrics tracking the loss
    loss=cce(y, y_pred)
    # Update the metrics.
    self.compiled_metrics.update_state(x,y_pred)
    # Return a dict mapping metric names to current value.
    # Note that it will include the loss (tracked in self.metrics).
    return {m.name:m.result() for m in self.metrics}

            

In [ ]:
def get_model(initializer = "uniform", latent_dim = 2048, num_heads = 8):
  global MAX_SEQ_LENGHT, VOCAB_SIZE, EMBEDDING_DIM,VECTORIZER
  encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
  x = PositionalEmbedding(MAX_SEQ_LENGHT, VOCABULARY_SIZE, EMBEDDING_DIM, initializer)(encoder_inputs)
  encoder_outputs = TransformerEncoder(EMBEDDING_DIM, latent_dim, num_heads)(x)
  encoder = keras.Model(encoder_inputs, encoder_outputs)

  decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
  encoded_seq_inputs = keras.Input(shape=(None, EMBEDDING_DIM), name="decoder_state_inputs")

  x = PositionalEmbedding(MAX_SEQ_LENGHT, VOCABULARY_SIZE, EMBEDDING_DIM, initializer)(decoder_inputs)
  x = TransformerDecoder(EMBEDDING_DIM, latent_dim, num_heads)(x, encoded_seq_inputs)
  x = layers.Dropout(0.5)(x)
  decoder_outputs = layers.Dense(VOCABULARY_SIZE, activation="softmax")(x)
  decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

  decoder_outputs = decoder([decoder_inputs, encoder_outputs])
  return CustomModel(
      [encoder_inputs, decoder_inputs], [decoder_outputs,encoder_outputs], name="transformer")

In [ ]:
LATENT_DIM = 2048
N_HEADS = 8
INITIALIZER = keras.initializers.Constant(EMBEDDING_MATRIX)

transformer = get_model(INITIALIZER, LATENT_DIM, N_HEADS)

transformer.summary()

In [ ]:
checkpoint_path='/content/drive/MyDrive/UniBO/NLP/training_2/'
EPOCHS = 10
tf.config.run_functions_eagerly(True)
callbacks = [
      EarlyStopping(patience=3, verbose=1),
      ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.00001, verbose=1),
      ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
  ]
transformer.compile(Adam(learning_rate = 1e-4), loss=return_loss(), metrics=[GULPEASE_index(),Simplification_loss()])

history = transformer.fit(TRAIN_DATASET, 
                            epochs=EPOCHS, 
                            validation_data=TEST_DATASET,
                            callbacks=callbacks
                            )